# Scraping

In [341]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
chrome_path = r'/Users/hongfan/chromeDriver/chromedriver'

In [342]:
#Chrome setting. Just ignore this
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/hongfan/user-generated-content-analytics'}
options.add_experimental_option('prefs', prefs)

In [343]:
driver = webdriver.Chrome(chrome_path, chrome_options=options) 

In [344]:
url="https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p"

In [345]:
import time
#scraping pages from n to m
def scrape_pages(n,m):
    a=[]
    c=[]
    d=[]
    q=[]
    for i in range(n, m):
        driver.get(url+ str(i))
        #time.sleep(1)
        author_list = driver.find_elements_by_class_name('Author')
        all_list = driver.find_elements_by_xpath("//div[@class='Message userContent']")
        post_list = driver.find_elements_by_xpath("//div[@class='Message userContent']/p[not(contains(@class, 'QuoteText'))]")
        date_list = driver.find_elements_by_xpath("//span[@class='MItem DateCreated']")
        for j in range(min(len(post_list),len(author_list),len(date_list))):
            date=date_list[j].text
            d.append(date)
            author= author_list[j].text
            a.append(author)
            #if you want to extract the quote from comment:
            try:
                quote=all_list[j].find_element_by_tag_name('blockquote').text 
            except NoSuchElementException:
                quote='No Quote'
            q.append(quote)
            comment = post_list[j].text
            c.append(comment)
    return d,a,q,c



In [354]:
data=scrape_pages(202,203)

In [355]:
import pandas as pd
data=pd.DataFrame({'date':data[0],'author':data[1],'quote':data[2],'comment':data[3]})

In [356]:
data

,date,author,quote,comment


In [321]:
#Save the dataframe as csv file
filename=driver.find_element_by_tag_name('h1').text
data.to_csv(filename+".csv")

In [357]:
df=pd.read_csv(filename+".csv")

In [358]:
df

,Unnamed: 0,date,author,quote,comment
0,0,June 2014,Kirstie_H,No Quote,"This is the all-new continuation of ""Stories f..."
1,1,June 2014,graphicguy,No Quote,I'm 2nd post....yee haw!!!!!!!!
2,2,June 2014,bwia,No Quote,Do I get a badge?
3,3,June 2014,imidazol97,"@bwia said:\nGreat, but will it be possible to...","Great, but will it be possible to refer to old..."
4,4,June 2014,stickguy,No Quote,And one more thing how do I find and add a dis...
5,5,June 2014,bwia,@stickguy said:\nif you want to bookmark this ...,"If you have posted in that discussion, click o..."
6,6,June 2014,bwia,@imidazol97 said:\nIf you have posted in that ...,If you are looking for a brand or model or a k...
7,7,June 2014,Kirstie_H,@graphicguy said:\nI'm 2nd post....yee haw!!!!...,"if you want to bookmark this one, there is a l..."
8,8,June 2014,cdnpinhead,No Quote,"Thanks @stickguy, it worked like a charm."
9,9,June 2014,graphicguy,"@Kirstie_H said:\nNo, but it's almost like you...",Thanks imidazol97. I will try that later but s...


In [359]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hongfan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: cannot use a string pattern on a bytes-like object

In [361]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(df["comment"])

TypeError: expected string or bytes-like object